In [ ]:
import yfinance as yf
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

# Ações para análise
acoes = ['AAPL', 'MSFT']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask com Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Gráfico de Candles das Ações AAPL e MSFT"),
    dcc.Graph(id='candlestick-plot')
])

@app.callback(
    Output('candlestick-plot', 'figure'),
    [Input('candlestick-plot', 'relayoutData')]
)
def update_candlestick_plot(relayout_data):
    return {
        'data': [
            go.Candlestick(
                x=dados_acoes.index,
                open=dados_acoes[acao]['Open'],
                high=dados_acoes[acao]['High'],
                low=dados_acoes[acao]['Low'],
                close=dados_acoes[acao]['Adj Close'],
                name=acao
            )
            for acao in acoes
        ],
        'layout': {
            'title': 'Gráfico de Candles das Ações AAPL e MSFT',
            'xaxis': {'title': 'Data'},
            'yaxis': {'title': 'Preço de Fechamento'}
        }
    }

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import yfinance as yf
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask com Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Gráfico de Candles da Ação do Facebook (FB)"),
    dcc.Graph(
        id='candlestick-plot',
        figure={
            'data': [
                go.Candlestick(
                    x=acao_facebook.index,
                    open=acao_facebook['Open'],
                    high=acao_facebook['High'],
                    low=acao_facebook['Low'],
                    close=acao_facebook['Close'],
                    name='FB'
                )
            ],
            'layout': {
                'title': 'Gráfico de Candles da Ação do Facebook (FB)',
                'xaxis': {'title': 'Data'},
                'yaxis': {'title': 'Preço'},
                'yaxis2': {'title': 'Volume', 'overlaying': 'y', 'side': 'right'}
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from flask import Flask, render_template

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o gráfico de candles da ação do Facebook
@app.route('/')
def plot_candlestick():
    fig = go.Figure(data=[go.Candlestick(x=acao_facebook.index,
                                        open=acao_facebook['Open'],
                                        high=acao_facebook['High'],
                                        low=acao_facebook['Low'],
                                        close=acao_facebook['Close'])])

    fig.update_layout(title='Gráfico de Candles da Ação do Facebook (FB)',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis2_title='Volume',
                      yaxis2=dict(overlaying='y', side='right'))

    return render_template('candlestick_plot.html', plot=fig.to_html())

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from flask import Flask, render_template

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o gráfico de candles da ação do Facebook
@app.route('/')
def plot_candlestick():
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Gráfico de candles
    fig.add_trace(go.Candlestick(x=acao_facebook.index,
                                 open=acao_facebook['Open'],
                                 high=acao_facebook['High'],
                                 low=acao_facebook['Low'],
                                 close=acao_facebook['Close'],
                                 name='FB'), row=1, col=1)

    # Gráfico de volume
    fig.add_trace(go.Bar(x=acao_facebook.index,
                         y=acao_facebook['Volume'],
                         name='Volume'), row=2, col=1)

    fig.update_layout(title='Gráfico de Candles e Volume da Ação do Facebook (FB)',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis2_title='Volume',
                      showlegend=False)

    return render_template('candlestick_plot.html', plot=fig.to_html())

if __name__ == '__main__':
    app.run(debug=True)


In [3]:
import yfinance as yf
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.models.widgets import Select
from datetime import datetime

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

# Criando a fonte de dados para o gráfico
source = ColumnDataSource(dados_acoes['Close'])

# Criando o gráfico de linha para cada ação
p = figure(x_axis_type='datetime', title='Preço de Fechamento das Ações', plot_width=800, plot_height=400)
for acao in acoes:
    p.line(x='Date', y=acao, source=source, line_width=2, legend_label=acao)

# Adicionando as ferramentas de interação
hover = HoverTool()
hover.tooltips = [("Data", "@Date{%F}"), ("Preço de Fechamento", "$y")]
hover.formatters = {'@Date': 'datetime'}
p.add_tools(hover)

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

# Criando o menu dropdown para selecionar a ação
menu = Select(options=acoes, value='AAPL', title='Selecione uma Ação')

# Função para atualizar o gráfico ao selecionar uma ação no menu dropdown
def update_plot(attr, old, new):
    selected_acao = menu.value
    p.title.text = f"Preço de Fechamento da Ação {selected_acao}"
    source.data = ColumnDataSource(dados_acoes[selected_acao]).data

menu.on_change('value', update_plot)

# Organizando o layout do dashboard
layout = column(menu, p)

# Configurando o documento do Bokeh
curdoc().add_root(layout)


ModuleNotFoundError: No module named 'bokeh'